# Material Classes

There are two material classes to use `mat_lib` and `mat_map`, they are both contained in `material.py`.

In [1]:
from material import mat_lib, mat_map
import numpy as np

1. [Readme](#README)
2. [API Summary](#API)


# README

## Mat_lib

`mat_lib` is the **Material Library**, and holds all the problem materials. This class should be used when info about a specific material, or all the materials is required. 

To initialize, it requires a list of all the desired material `xml` files, and the number of energy groups in the desired group structure, `n_grps`:

In [2]:
file_loc = './tests/testData/materials/'
files = [file_loc + m for m in ['test_mat.xml', 'test_mat2.xml']]
lib = mat_lib(n_grps = 2, files = files)

The library can also be intialized with only the number of groups, and then materials added using `mat_lib.add`, this is probably more useful if iterating through all the files in a directory. `n_grps` is always required at initialization.

In [3]:
lib2 = mat_lib(n_grps = 2)
for f in files:
    lib2.add(f)

Two utility functions return info about the materials, `ids` returns the material IDs and `props` shows all the properties that each material has, or the properties of a specific material id.

In [4]:
lib.ids()

['test_mat', 'test_mat2']

In [5]:
lib.props()

{'test_mat': [['name', 'id'],
  ['nu'],
  ['chi', 'g_thermal'],
  ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
  ['diff_coef_ua',
   'diff_coef',
   'chi_nu_sig_f',
   'inv_sig_t',
   'sig_t_ua',
   'ksi_ua']],
 'test_mat2': [['name', 'id'],
  ['nu'],
  ['chi', 'g_thermal'],
  ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
  ['diff_coef_ua',
   'diff_coef',
   'chi_nu_sig_f',
   'inv_sig_t',
   'sig_t_ua',
   'ksi_ua']]}

In [6]:
lib.props('test_mat')

[['name', 'id'],
 ['nu'],
 ['chi', 'g_thermal'],
 ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
 ['diff_coef_ua',
  'diff_coef',
  'chi_nu_sig_f',
  'inv_sig_t',
  'sig_t_ua',
  'ksi_ua']]

#### Accessing Data

The main method to access data is `get`. This will either return:
- A dictionary containing all the materials as keys, and the desired property as values **or**
- The value of the property for a particular material.

In [7]:
# Get dictionary of all sig_t values:
lib.get('sig_t')

{'test_mat': array([ 20.,  30.]), 'test_mat2': array([ 10.,  20.])}

In [8]:
# Get values for a specific material:
lib.get('sig_t', mat_id = 'test_mat')

array([ 20.,  30.])

Note this is identical to using:

In [9]:
lib.get('sig_t')['test_mat']

array([ 20.,  30.])

To get a property per steradian, use `get_per_str`:

In [10]:
lib.get_per_str('sig_t', mat_id='test_mat')

array([ 1.59154943,  2.38732415])

Other functions will return cell corrections and bd corrections using the same syntax:

In [11]:
print lib.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]))
print lib.bd_correction_ua(np.array([1.0, 1.0]))

{'test_mat2': array([ 120.,  120.]), 'test_mat': array([ 70.,  70.])}
{'test_mat2': 120.0, 'test_mat': 70.0}


## MAT_MAP

`mat_map` is a **material mapping** that takes the problem material layout and connects it to the materials contained in the material library.

To make a mapping, it requires
- `lib` A material library that contains all problem materials
- `layout` A string showing the material layout of the problem, with different values meaning different materials. This must have a square number of values to map to the square problem.
- `layout_dict` a dictionary connecting the strings used in the layout to the material ids in the library.
- `x_max` maximum x-value of the problem.
- `n` number of cells along each side of the problem
OPTIONAL
- `x_min`, `y_min` otherwise defaults to 0
- `y_max` otherwise defaults to `x_max`

The layout string uses any characters to represent different materials, you can use any characters or strings that you want. For example, to have one material surrounded by another.

**Note:** The lower left of this array is the problem origin (0,0)

In [12]:
# Both of these are identical, 
# as any line breaks and extra spaces are removed. 
layout = """1 1 1 1
            1 2 2 1
            1 2 2 1
            1 1 1 1"""
layout2 = " 1 1 1 1 1 2 2 1 1 2 2 1 1 1 1 1"

Next, the arbirary characters used (1 and 2 here) need to be linked back to materials in your material library. For example, we have two materials in ours:

In [13]:
lib.ids()

['test_mat', 'test_mat2']

We link them using a dictionary, pairing the layout character with the material id:

In [14]:
layout_dict = {'1':'test_mat2', '2': 'test_mat'}

We can then make our mapping:

In [15]:
mmap = mat_map(lib = lib, layout = layout, layout_dict = layout_dict, 
               x_max = 4, n=4)

Now we can get material properties at any (x,y) value, or any cell index. Here, we get the name of the material, and the $\Sigma_t$ using x and y coordinates in a tuple:

In [16]:
print mmap.get('id', (0.5, 0.5))
print mmap.get('sig_t', (0.5, 0.5))

test_mat2
[ 10.  20.]


As expected, that location returns the correct material and properties. Cell index can also be used:

In [17]:
print mmap.get('id', 1)
print mmap.get('sig_t', 1)

test_mat2
[ 10.  20.]


Use the same names to get cell corrections:

In [18]:
mmap.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]), (0.5, 0.5))

array([ 120.,  120.])

# API

This is a summary of all functions.

## mat_lib

In [19]:
file_loc = './tests/testData/materials/'
files = [file_loc + m for m in ['test_mat.xml', 'test_mat2.xml']]

In [21]:
# Initialize using list of files
lib = mat_lib(n_grps = 2, files = files) 

# Add new ones using add
lib2 = mat_lib(n_grps = 2)
for f in files:
    lib2.add(f)

In [22]:
# Get IDs of materials
lib.ids()

['test_mat', 'test_mat2']

In [24]:
# Get Properties of materials
lib.props()

{'test_mat': [['name', 'id'],
  ['nu'],
  ['chi', 'g_thermal'],
  ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
  ['diff_coef_ua',
   'diff_coef',
   'chi_nu_sig_f',
   'inv_sig_t',
   'sig_t_ua',
   'ksi_ua']],
 'test_mat2': [['name', 'id'],
  ['nu'],
  ['chi', 'g_thermal'],
  ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
  ['diff_coef_ua',
   'diff_coef',
   'chi_nu_sig_f',
   'inv_sig_t',
   'sig_t_ua',
   'ksi_ua']]}

In [25]:
# Get properties of a specific material
lib.props('test_mat')

[['name', 'id'],
 ['nu'],
 ['chi', 'g_thermal'],
 ['sig_s', 'sig_a', 'sig_f', 'sig_t'],
 ['diff_coef_ua',
  'diff_coef',
  'chi_nu_sig_f',
  'inv_sig_t',
  'sig_t_ua',
  'ksi_ua']]

In [26]:
#Get properties of all materials
lib.get('sig_t')

{'test_mat': array([ 20.,  30.]), 'test_mat2': array([ 10.,  20.])}

In [27]:
#Get property of a specific material
lib.get('sig_t', mat_id = 'test_mat')

array([ 20.,  30.])

In [28]:
# Calculate corrections:
print lib.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]))
print lib.bd_correction_ua(np.array([1.0, 1.0]))

{'test_mat2': array([ 120.,  120.]), 'test_mat': array([ 70.,  70.])}
{'test_mat2': 120.0, 'test_mat': 70.0}


In [30]:
# can also get for a specific material
lib.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]), 
                             mat_id='test_mat')

array([ 70.,  70.])

## mat_map

In [31]:
# Create layout and layout dictionary
layout = """1 1 1 1
            1 2 2 1
            1 2 2 1
            1 1 1 1"""
layout_dict = {'1':'test_mat2', '2': 'test_mat'}

# Initialize
mmap = mat_map(lib = lib, layout = layout, layout_dict = layout_dict, 
               x_max = 4, n=4)

In [32]:
# Get values at locations, (x,y) or cell index k
print mmap.get('id', (0.5, 0.5))
print mmap.get('sig_t', (0.5, 0.5))
print mmap.get('id', 1)
print mmap.get('sig_t', 1)

test_mat2
[ 10.  20.]
test_mat2
[ 10.  20.]


In [34]:
# Get corrections at locations using the same syntax
mmap.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]), (0.5, 0.5))
mmap.cell_correction_ua(np.array([[1.0, 1.0],[1.0,1.0]]), 1)

array([ 120.,  120.])